In [ ]:
import numpy as np
from PIL import Image

# Image size
WIDTH, HEIGHT = 400, 300

# Camera parameters
CAM_POS = np.array([1.0, 0.4, 1.0])  # fixed camera position
CAM_TARGET = np.array([0.0, 0.0, 0.0])

# Ray marching parameters
MAX_STEPS = 256
MAX_DIST = 3.0
EPSILON = 1e-4

def normalize(v):
    return v / np.linalg.norm(v)

def dot(a, b):
    return np.dot(a, b)

def length(v):
    return np.linalg.norm(v)

def cross(a, b):
    return np.cross(a, b)


def sdCylinder(p, h, r):
    d = np.abs(np.array([length(p[[0,2]]), p[1]])) - np.array([r, h])
    return min(max(d[0], d[1]), 0.0) + length(np.maximum(d, 0.0))


def map_func(pos):
    # Use vertical cylinder with height=0.3 and radius=0.2 at origin
    return sdCylinder(pos, 0.3, 0.2)

# Calculate normal using central differences on the SDF
def calcNormal(pos):
    eps = 0.0005
    e = np.array([1.0, -1.0]) * 0.5773
    n = np.zeros(3)
    n[0] = e[0]*map_func(pos + np.array([e[0], e[1], e[1]]) * eps) + \
           e[1]*map_func(pos + np.array([e[1], e[0], e[0]]) * eps) + \
           e[1]*map_func(pos + np.array([e[1], e[0], e[1]]) * eps) + \
           e[0]*map_func(pos + np.array([e[0], e[0], e[0]]) * eps)
    n[1] = e[0]*map_func(pos + np.array([e[1], e[0], e[1]]) * eps) + \
           e[1]*map_func(pos + np.array([e[0], e[1], e[0]]) * eps) + \
           e[0]*map_func(pos + np.array([e[1], e[0], e[0]]) * eps) + \
           e[1]*map_func(pos + np.array([e[0], e[0], e[1]]) * eps)
    n[2] = e[0]*map_func(pos + np.array([e[1], e[1], e[0]]) * eps) + \
           e[1]*map_func(pos + np.array([e[0], e[1], e[1]]) * eps) + \
           e[0]*map_func(pos + np.array([e[0], e[1], e[0]]) * eps) + \
           e[1]*map_func(pos + np.array([e[1], e[0], e[1]]) * eps)
    return normalize(n)

def ray_march(ro, rd):
    t = 0.0
    for _ in range(MAX_STEPS):
        pos = ro + t*rd
        dist = map_func(pos)
        if dist < EPSILON:
            return t, pos
        if t > MAX_DIST:
            break
        t += dist
    return None, None

def render():
    aspect_ratio = WIDTH / HEIGHT
    
   
    forward = normalize(CAM_TARGET - CAM_POS)
    right = normalize(cross(forward, np.array([0,1,0])))
    up = cross(right, forward)
    
    image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.float32)
    
    for y in range(HEIGHT):
        for x in range(WIDTH):
            # Normalized pixel coordinates (from -1 to 1)
            px = (2*(x + 0.5)/WIDTH - 1) * aspect_ratio
            py = 1 - 2*(y + 0.5)/HEIGHT
            
            # Ray direction
            rd = normalize(px*right + py*up + 1.5*forward)
            
            t, pos = ray_march(CAM_POS, rd)
            col = np.array([0.0, 0.0, 0.0])
            
            if pos is not None:
                normal = calcNormal(pos)
                light_dir = normalize(np.array([0.57703, 0.57703, 0.57703]))  # directional light
                
                diff = max(dot(normal, light_dir), 0.0)
                amb = 0.5 + 0.5*max(dot(normal, np.array([0,1,0])), 0.0)
                col = np.array([0.2, 0.3, 0.4])*amb + np.array([0.8, 0.7, 0.5])*diff
                
                col = np.sqrt(np.clip(col, 0, 1))
            
            image[y, x] = col
    

    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    img = Image.fromarray(image, 'RGB')
    img.save('cylinder.png')
    print("done")
    

if __name__ == "__main__":
    render()